<a href="https://colab.research.google.com/github/gumendes/projeto_idosos_atividade_fisica/blob/main/02_dashboard_e_gamificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configurações visuais
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('pastel')

# Leitura da base
df = pd.read_excel('../content/dados_idosos_simulados_v2.xlsx')

# Converter data
df['data'] = pd.to_datetime(df['data'])
df['dia_semana'] = df['data'].dt.day_name()
df.head()


,id_participante,idade,sexo,data,atividade,presenca,clima,duracao_min,satisfacao,dia_semana
0,1,73,F,2025-08-01,Caminhada,0,Frio,0,0,Friday
1,1,73,F,2025-08-04,Ginástica,0,Frio,0,0,Monday
2,1,73,F,2025-08-06,Alongamento,1,Frio,39,5,Wednesday
3,1,73,F,2025-08-08,Jogo com bola,1,Frio,42,4,Friday
4,1,73,F,2025-08-11,Ginástica,1,Frio,29,3,Monday


In [ ]:
# Pontuação base: 10 pontos por presença
df['pontos'] = df['presenca'] * 10

# Bônus por satisfação alta
df.loc[df['satisfacao'] >= 4, 'pontos'] += 5

# Bônus por frequência contínua (presenças seguidas)
df['presenca_shift'] = df.groupby('id_participante')['presenca'].shift(1)
df['bonus_consistencia'] = np.where((df['presenca'] == 1) & (df['presenca_shift'] == 1), 3, 0)
df['pontos'] += df['bonus_consistencia']

# Pontuação total por participante
ranking = df.groupby('id_participante')['pontos'].sum().reset_index().sort_values('pontos', ascending=False)
ranking.head(10)


,id_participante,pontos
6,7,310
17,18,299
11,12,286
0,1,276
10,11,275
1,2,271
5,6,263
4,5,247
14,15,247
16,17,247


In [ ]:
fig = px.bar(
    ranking,
    x='id_participante',
    y='pontos',
    title='🏆 Ranking de Participação — Sistema de Pontos',
    text='pontos',
    color='pontos',
    color_continuous_scale='Blues'
)
fig.update_layout(xaxis_title="Participante", yaxis_title="Total de Pontos")
fig.show()


In [ ]:
presenca_tempo = df.groupby('data')['presenca'].mean().reset_index()

fig = px.line(
    presenca_tempo,
    x='data',
    y='presenca',
    title='📈 Evolução da Presença ao Longo do Tempo',
    markers=True
)
fig.update_yaxes(title='Taxa de Presença (%)', range=[0,1])
fig.show()


In [ ]:
fig = px.box(
    df,
    x='atividade',
    y='satisfacao',
    color='presenca',
    title='🎯 Satisfação por Atividade e Presença'
)
fig.update_layout(xaxis_title="Atividade", yaxis_title="Satisfação (1 a 5)")
fig.show()


In [ ]:
presenca_clima = df.groupby('clima')['presenca'].mean().reset_index()
presenca_dia = df.groupby('dia_semana')['presenca'].mean().reset_index()

fig1 = px.bar(presenca_clima, x='clima', y='presenca', title='☀️ Presença Média por Clima', color='presenca', color_continuous_scale='Teal')
fig2 = px.bar(presenca_dia, x='dia_semana', y='presenca', title='📅 Presença Média por Dia da Semana', color='presenca', color_continuous_scale='Purples')

fig1.show()
fig2.show()


In [ ]:
# Estatísticas gerais
presenca_geral = df['presenca'].mean() * 100
media_satisf = df['satisfacao'].mean()
atividade_top = df.groupby('atividade')['presenca'].mean().idxmax()

print(f"📊 Taxa média de presença: {presenca_geral:.1f}%")
print(f"💬 Satisfação média geral: {media_satisf:.2f}")
print(f"🏅 Atividade com maior presença: {atividade_top}")

# Gráfico de dispersão: Satisfação vs Idade
fig = px.scatter(
    df,
    x='idade',
    y='satisfacao',
    color='presenca',
    title='💡 Relação entre Idade, Satisfação e Presença',
    hover_data=['atividade', 'clima', 'data']
)
fig.show()


📊 Taxa média de presença: 63.3%
💬 Satisfação média geral: 2.42
🏅 Atividade com maior presença: Caminhada


In [ ]:
# Salvar o ranking e pontuações para integração com o Power BI ou painel web
ranking.to_csv('../data/ranking_gamificacao.csv', index=False)
print("✅ Arquivo salvo: ../data/ranking_gamificacao.csv")


OSError: Cannot save file into a non-existent directory: '../data'